# <div style = "text-align: left">  <br><span style = "font-family:Roboto Thin; font-size:1.5em; color:#FF5000">sisu</span> Task List Processing <span style = "color:#FF5000">Version 0.0 </span></div>

<div style = "text-align: left"> The purpose of this notebook is to process, and reformat,
                                   Task List information, <br> brought in as an excel document,
                                   down to SQL statements for inserting into <span style = "font-family:Roboto Thin; font-size:1em; color:#FF5000">sisu</span>. </div>

## 0 <span style="color:#b54dff">Library

In [1]:
import os
from termcolor import colored
import pwd # needed if the document is stored on computer instead of Google Drive
import tkinter as tk
from tkinter import filedialog
import pandas as pd
pd.set_option('display.max_columns', None) # keeps pandas from truncating columns
import numpy as np
import clipboard
pd.options.display.max_colwidth = 500
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate
run_list = []

## 1 <span style="color:#8b49fc"> Select File, View Information and Clean

<span style = "color:#73efff"> Make sure that the file you are working with is in a *FOLDER* that is formated with the team ID, a space, and then the team name. For example: 12345 Pretend Realty

In [2]:
# Cell Name: FILE

'''
This cell is used to access the template file that we want to work with.

This code was built for MacOS, but I will eventually set it up for Windows also.
It checks if you have Google Drive Desktop App installed, and if so will open a path to the IMPORTS
folder when looking for the file, otherwise it will open the 'Downloads' folder in the Finder. 

If you have Google Drive Destop installed, but prefer to save files to your computer, set the check_for_google_drive
variable to False (True and False are case sensitive). 
'''

check_for_google_drive = True
local_folder = "Downloads" #you can change this to start at a different folder. 

root = tk.Tk()
root.withdraw()

file_path = '' #filedialog.askopenfilename()

if check_for_google_drive:
    if 'Google Drive.app' in os.listdir("/Applications/"):
        root.file_path =  filedialog.askopenfilename(initialdir = "/Volumes/GoogleDrive/My Drive/IMPORTS",title = "SELECT the Task List File")
    else:
        root.file_path =  filedialog.askopenfilename(initialdir = "/Users/"+pwd.getpwuid(os.getuid()).pw_name+"/{local_folder}",title = "SELECT the CSV file from Jira")
else:
    root.file_path =  filedialog.askopenfilename(initialdir = "/Users/"+pwd.getpwuid(os.getuid()).pw_name+"/{local_folder}",title = "SELECT the CSV file from Jira")


# print ('Source file: ' + root.file_path)

'''
This cell formats the import as a dictionary of DataFrames, and also collects other important bits of data.

For this to work properly, the name of the folder that the file is in must lead with the team ID.
'''

team_id = root.file_path.split("/")[-2].split(" ")[0] 
team_name = " ".join(root.file_path.split("/")[-2].split(" ")[1:])
file_name = root.file_path.split("/")[-1]
xl_file = pd.read_excel(root.file_path, sheet_name = None)
xl_sheets = list(xl_file.keys())
'''
Sanity check to make sure that the information is collected correctly, and to give an overview of 
what is present in the file. 
'''

print(f"Team ID:\t{team_id}")
print(f"Team Name:\t{team_name}")
print(f"File Name:\t{file_name}")
for i in range(0, len(xl_sheets)):
    print(f"Sheet {i+1} (i={i}):   \t{xl_sheets[i]}")
print(' ')

# If the first column Header is 'Task List Name' then we need to capture this sheet. This will capture some sheets with no info, but later cleaning will remove that data. 

task_sheet_names = [xl_sheets[i] for i in range(0, len(xl_sheets)) if xl_file[xl_sheets[i]][xl_file[xl_sheets[i]].columns[0]][0] == 'Task List Name']
if len(task_sheet_names) > 0:
    # print('Task list template sheets present.')
    pass
else:
    print(colored("ERROR: No task list template sheets present.", 'red', attrs=['bold']))
    print(' ')

# Fix the headers
# The header for this data is all unknown (because Row one in the template isn't the header). Because row 2 (row 2 in the CSV mind you) is the header, we are setting that row as the header. 

for i in task_sheet_names:
    xl_file[i].columns = xl_file[i].iloc[0]
    xl_file[i] = xl_file[i].drop(0, axis=0).reset_index(drop=True)
    if xl_file[i].columns[0] == 'Task List Name':
        # 
        pass
    else:
        print(colored(f'ERROR: {i} is not valid.', 'red', attrs=['bold']))
print(' ')


# Combine the sheets
# We have a list of all the sheets that we will use (task_sheet_name) which we will use to combine all the sheets into a single DataFrame (df)
print("SHEETS WITH TASKS AND COUNT OF TASKS/ROWS:")
cols = xl_file[task_sheet_names[0]].columns
df = pd.DataFrame(columns = cols)

sheet_len = 0
for i in task_sheet_names:
    xl_file[i]['Task Name'] = xl_file[i]['Task Name'].replace({'':np.nan})
    xl_file[i] = xl_file[i][xl_file[i]['Task Name'].notna()]
    xl_file[i]['sheet_name'] = i
    df = df.append(xl_file[i])
    file_len = len(xl_file[i][1:])
    print(colored(file_len,'cyan') + f" rows in {i}")
    sheet_len = sheet_len+file_len

print(colored(sheet_len,'cyan')+ " rows in total.")
print(' ')

#Clean combined sheets
# Some of this cleaning is redundant (but better safe than sorry). Removing the csv indicators for the end of list and instructions. Also, if the Task name is empty, then the row will be removed. 

df = df[(df['Task List Name']!='Task List Name') & (df['Task List Name'].str.contains('end of list|Enter your task')==False) & (df['Task Name'].notna())]
df = df.fillna('')
df['Days'] = df['Days'].replace({'': 0})
print(colored('All empty Day values have been set to zero.', 'green', attrs=['bold']))


# This strips all the whitespace from around the strings in the DataFrame.
for i in df.columns:
    '''
    Simple function says
        1. look at column
        2. if column is string, strip whitespace
        3. move to next column
        4. repeat until all columns checked
    '''
    if df[i].dtype == "O":    # strings in a DataFrame are called objects, and the dtype 'Object' of a column in a DataFrame is presented as 'O'  
        df[i] = df[i].str.strip() 
        df[i] = df[i].str.replace("UPDATE", "UP-DATE")
        df[i] = df[i].str.replace("update", "up-date")
        df[i] = df[i].str.replace("'", "")
        df[i] = df[i].str.replace('"', "")
        df[i] = df[i].str.replace(r'”', "") # special quotations
        df[i] = df[i].str.replace(r'“', "") # special quotations
        df[i] = df[i].str.replace("\r", " - ")
        df[i] = df[i].str.replace("\n", "")
        df[i] = df[i].str.replace(chr(13), " - ")
        df[i] = df[i].str.replace(chr(10), "")
        df[i] = df[i].str.replace('etc…', 'etc') # The elipses is a character that SISU doesn't acknowledge and drops

    

print(colored('All whitespace cleared from string values.', 'green', attrs=['bold']))
print(' ')



'''
Alternate method for stripping the whitespace:

df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

'''

def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df




df = df.reset_index()
# Validator For Task List Template

# Columns
task_list_template_validation_cols = ['Task List Name', 'List Description',
       'List descr. remaining\ncharacters', 'Applies to Buyer/Seller',
       'Buyer/Seller code', 'Task Name', 'Task name remaining\ncharacters',
       'Task Description', 'Task description remaining\ncharacters',
       'Task Trigger date \n(Relative due date)', 'Trigger Date DB (Sisu)',
       'Assign to TC, Agent or assignee full name', 'Assign To T/A/Agent ID',
       'Days', 'Task or Notification?']
for i in task_list_template_validation_cols:
    if i in df.columns:
        pass
    else:
        print(colored(f"ERROR: Required column {i} not found in template.", 'red', attrs = ['bold']))
    

df['Assign to TC, Agent or assignee full name'] = df['Assign to TC, Agent or assignee full name'].replace({'':np.nan})
if len(df[(df['Assign to TC, Agent or assignee full name'].isna()) & (df['Task or Notification?'] == 'T')])>0:
    print(colored('THE FOLLOWING TASKS ARE MISSING ASSINEE:', 'red', attrs=['bold']) + f" TOTAL: {len(df[df['Assign to TC, Agent or assignee full name'].isna()])}")
    with pd.option_context("display.max_rows", 1000):  
        print(tabulate(df[df['Assign to TC, Agent or assignee full name'].isna()][['sheet_name','Task Name']]))

df['Task or Notification?'] = df['Task or Notification?'].replace({'':np.nan})
if len(df[(df['Task or Notification?'].isna()) | (df['Task or Notification?'].str.contains('T|N')==False)])>0:
    print(colored('THE FOLLOWING TASKS ARE MISSING, OR HAVE INCORRECT, "Task or Notification?" INFORMATION:', 'red', attrs=['bold']) + f" TOTAL: {len(df[(df['Task or Notification?'].isna()) | (df['Task or Notification?'].str.contains('T|N')==False)])}")
    with pd.option_context("display.max_rows", 1000):
        print(tabulate(df[(df['Task or Notification?'].isna()) | (df['Task or Notification?'].str.contains('T|N')==False)][['sheet_name','Task Name']]))


# Excel won't allow for duplicate sheet names. 



if len(df[df[['Task List Name', 'Task Name']].duplicated()]) !=0:
    print(colored(f"WARNING: ", 'yellow') + f"{len(df[df[['Task List Name', 'Task Name']].duplicated()])} of {len(df)} Task List Name and Task Name combinations are duplicates.")
    print('')

if len(df['Task List Name'].unique()) != len(df['sheet_name'].unique()):
    mask_task = np.isin(df['Task List Name'].unique(), df['sheet_name'].unique())
    sheet_mask = np.isin(df['sheet_name'].unique(), df['Task List Name'].unique())
    print(colored(f"WARNING: ", 'yellow') + "Some of the Task List Names do not match the sheet names.")
    print(f"The Task Lists Names: {df['Task List Name'].unique()[~mask_task]} are not among the sheet names.")
    print(f"The sheet names: {df['sheet_name'].unique()[~sheet_mask]} are not among the Task List Names.")

print(" ")
print(colored(f'select Name, Team_id, Status from team where Team_id = {team_id}', 'green') + ' has been add to your clipboard\nPaste into the Raw Data Tool to validate that we are working with the correct Team ID.\nVerify that the Team ID matches the ticket in JIRA.')
clipboard.copy(f'select Name, Team_id, Status from team where Team_id = {team_id}')


run_list.append('1. FILE')

Team ID:	16032
Team Name:	Freedom Reps
File Name:	16032 FREEDOM_ONBOARDING Task List Template V9.7 YR V4.xlsx
Sheet 1 (i=0):   	Pre Agent-Onboarding
Sheet 2 (i=1):   	New Agent
Sheet 3 (i=2):   	Experienced Agent
Sheet 4 (i=3):   	TJ New Agent
Sheet 5 (i=4):   	TJ Experienced Agent
Sheet 6 (i=5):   	Team Contract Signing 
Sheet 7 (i=6):   	Team Pre-Onboarding
Sheet 8 (i=7):   	Team Onboarding
Sheet 9 (i=8):   	Data mapping (hidden)
 
 
SHEETS WITH TASKS AND COUNT OF TASKS/ROWS:
16 rows in Pre Agent-Onboarding
28 rows in New Agent
30 rows in Experienced Agent
36 rows in TJ New Agent
37 rows in TJ Experienced Agent
10 rows in Team Contract Signing 
19 rows in Team Pre-Onboarding
22 rows in Team Onboarding
198 rows in total.
 
All empty Day values have been set to zero.
All whitespace cleared from string values.
 
THE FOLLOWING TASKS ARE MISSING ASSINEE: TOTAL: 28
--  ---------  ---------------------------------------------------------------------------------------------------------------

In [3]:
df.loc[df['sheet_name']=='New Agent', 'Assign to TC, Agent or assignee full name'] = 'A'

In [57]:
# Tool for the scenario when the client named all the sheets correctly but did not name the Task Lists correctly. The tool will correct the Task List Name. 
# To use, set the use_sheet_names_as_task_list_names variable to True. 

use_sheet_names_as_task_list_names = False
if use_sheet_names_as_task_list_names:
    df['Task List Name'] = df['sheet_name']

## 2 <span style = "color:#577eff"> Get </span> *Task Lists* <span style = "color:#577eff"> that already exist in </span> <span style = "font-family:Roboto Thin; font-size:1em; color:white">sisu</span>  

In [5]:
# Cell Name: Task List From Sisu

task_list_sql_text = f"select * from client_task_list where team_id = {team_id} AND status = 'N'  ORDER BY task_list_id;"
df_task_list_sql_text = pd.DataFrame([task_list_sql_text])
df_task_list_sql_text.to_clipboard(index=False,header=False)

print(colored(f"Collecting Task Lists from Sisu for team {team_id}", attrs = ['bold']))
print(colored("SQL query has been copied to your clipboard. \nPaste this into Sisu's Raw Data Tool", 'green', attrs=['bold']))
print(colored("Copy the returned table from the Raw Data Tool and run the following cell.", 'cyan', attrs = ['bold']))

run_list.append('2. Task List From Sisu')

SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.


In [6]:
# Cell Name: Insert Task Lists

if 'df_reset_1' in locals():
    pass
else:
    df_reset_1 = df

'''Once the SQL query information for the existing Task Lists is copied to the clipboard, run this cell'''

# if 'current_task_list_names' in locals():
#     pass
# else:
current_task_list_names = pd.read_clipboard()
if current_task_list_names.columns == '0': # If there are no task lists yet, the quere will return 0, in which case we need to start a blank DataFrame with the correct column names
    current_task_list_names = pd.DataFrame(columns = ['task_list_id', 'team_id', 'name', 'dscr', 'client_type_id', 'created_ts', 'updated_ts', 'status', 'display_order', 'status_trigger', 'trigger_by', 'transaction_stage_trigger'])
else:
    pass

print("TASK LIST COUNT")
print(colored(f"{len(current_task_list_names)} ", 'cyan') + "Current Task Lists")
print(colored(f"{len(df['Task List Name'].unique())} ",'cyan') + "New Task Lists")
final_task_list_count = len(current_task_list_names)+len(df['Task List Name'].unique())
print(colored(f"{final_task_list_count} ", 'cyan') + "Total Task Lists")
# Clean and add columns for the SQL insert.

df['Team ID'] = team_id #from the folder that the file is in
df['created_ts'] = 'current_timestamp'
df['updated_ts'] = 'NULL'
df['Status'] = 'N'
# df['display_order'] = [current_task_list_names['display_order'].max()+ 1 + i for i in range(0, len(df))]
df['status_trigger'] = ''


# list and arrange columns for client task list SQL insert (This layout is from the VBA excel tool -- Sheet: )
client_task_list_cols = ['Team ID', 'Task List Name', 'List Description', 'Buyer/Seller code', 'created_ts', 'updated_ts', 'Status', 'status_trigger']

# Replace NaN values with ''
df_client_task_list = df[client_task_list_cols].fillna('')
df_client_task_list = df_client_task_list[-df_client_task_list['Task List Name'].duplicated()].reset_index(drop = True)
if len(current_task_list_names['display_order']) > 0: # If there are no current task lists, we start the display order at 1
    df_client_task_list['display_order'] = [current_task_list_names['display_order'].max()+ 1 + i for i in range(0, len(df_client_task_list))]
else:
    df_client_task_list['display_order'] = [1 + i for i in range(0, len(df_client_task_list))]
client_task_list_cols_order = ['Team ID', 'Task List Name', 'List Description', 'Buyer/Seller code', 'created_ts', 'updated_ts', 'Status', 'display_order', 'status_trigger']

# Check if the display_order is interacting as expected by checking that the final Task List's display_order number matches the expected total number of task lists. 
if df_client_task_list['display_order'].iloc[-1] == final_task_list_count:
    # print("Finals display order equals expected Final Task List count.")
    pass
else:
    print(colored('ERROR: Final display order does NOT equal the expected Final Task List count.', 'red', attrs=['bold']))

    
# Create SQL insert statement, and save to clipboard
task_list_values = "("+df_client_task_list['Team ID']+",'"+df_client_task_list['Task List Name']+"', '"+df_client_task_list['List Description']+"', '"+df_client_task_list['Buyer/Seller code']+"', "+df_client_task_list['created_ts']+", "+df_client_task_list['updated_ts']+", '"+df_client_task_list['Status']+"', "+df_client_task_list['display_order'].astype(str)+", '"+df_client_task_list['status_trigger']+"'),"
df_client_task_list['task_list_values']  = task_list_values
task_list_insert_statement ='INSERT INTO client_task_list ("team_id","name","dscr","client_type_id","created_ts","updated_ts","status","display_order","status_trigger") \nVALUES'

string = df_client_task_list['task_list_values'].to_string(header= False, index = False)
while "  " in string:
    string = string.replace("  ", " ")

clipboard.copy(task_list_insert_statement + "\n" +  string.replace("\n (", "\n(").strip()[:-1] + ";")
# clipboard.copy(task_list_insert_statement + "\n" +  df_client_task_list['task_list_values'].to_string(index = False, header = False).strip()[0:-1].replace("     ", " ").replace("    ", " ").replace('   ', ' ').replace('  ', ' ').replace(' (',  '(') + ";")
df['Task List Name'] = df['Task List Name'].replace('   ','').replace('  ', '')

print(' ')
print(colored(f"Inserting Task List from the Team {team_id} template", 'white', attrs = ['bold']))
print(colored(f"An INSERT statement for the Task List from the team {team_id} template has been copied to your clipboard. \nPaste into the Sisu Raw Data Tool \nThis will load the Task Lists from the template into Sisu.", 'green', attrs=['bold']))

run_list.append('3. Insert Task Lists')

TASK LIST COUNT
0 Current Task Lists
8 New Task Lists
8 Total Task Lists
 
Inserting Task List from the Team 16032 template
An INSERT statement for the Task List from the team 16032 template has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Task Lists from the template into Sisu.


## 3 <span style="color:#40a0bd"> Get </span>  *Tasks* <span style="color:#40a0bd"> from </span> *client_task_blueprint*

In [8]:
# Cell Name: Tasks From Sisu

task_blueprint_sql_text = f"select * from client_task_blueprint where team_id = {team_id} and status = 'N' order by task_blueprint_id;"
df_task_blueprint_sql_text = pd.DataFrame([task_blueprint_sql_text])
df_task_blueprint_sql_text.to_clipboard(index=False, header=False)

print(colored(f"Collecting Task Blueprints from Sisu for team {team_id}", 'white', attrs = ['bold']))
print(colored("SQL query has been copied to your clipboard. \nPaste this into Sisu's Raw Data Tool", 'green', attrs=['bold']))
print(colored("Copy the returned table from the Raw Data Tool and run the following cell.", 'cyan', attrs = ['bold']))

run_list.append('4. Tasks From Sisu')

SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.


In [9]:
# Cell Name: Process Tasks and Get Agent Info From Sisu

'''Once SQL query for the task blueprint is copied to the clipboard, run this cell'''

current_task_blueprint = pd.read_clipboard()
if current_task_blueprint.columns == '0':
    current_task_blueprint = pd.DataFrame(columns = ['task_blueprint_id', 'team_id', 'name', 'dscr', 'task_type', 'display_order', 'related_client_date_column', 'due_days', 'status', 'client_type_id', 'created_ts', 'updated_ts', 'assign_to', 'email_template_id', 'email_subject', 'email_recipients'])
else:
  pass

print("TASK COUNT")
print(colored(f"{len(current_task_blueprint)} ", 'cyan') + "Current Tasks")
print(colored(f"{len((df['Task List Name'] + ' ' +  df['Task Name']).unique())} ", 'cyan') + "New Tasks")
final_task_name_count = len(current_task_blueprint)+len((df['Task List Name'] + ' ' +  df['Task Name']).unique())
print(colored(f"{final_task_name_count} ", 'cyan') + "Total Tasks")

'''Task names, types, display_orders, and descriptions'''
client_task_blueprint_cols = ['Team ID', 'Task Name', 'Task Description', 'Task or Notification?', 'display_order', 'related_client_date_column', 'Days', 'Status', 'client_type_id', 'created_ts', 'updated_ts', 'assign_to']


# Get agent information

agent_info_sql_text = f"select a.first_name, a.last_name, a.agent_id from team t left join team_agent ta on t.team_id = ta.team_id left join agent a on ta.agent_id = a.agent_id where ta.team_id = {team_id} and a.status = 'N';"
df_agent_info_sql_text = pd.DataFrame([agent_info_sql_text])
df_agent_info_sql_text.to_clipboard(index=False,header=False)

print(colored(f"Collecting Agent Information from Sisu for team {team_id}", 'white', attrs = ['bold']))
print(colored("SQL query has been copied to your clipboard. \nPaste this into Sisu's Raw Data Tool", 'green', attrs=['bold']))
print(colored("Copy the returned table from the Raw Data Tool and run the following cell.", 'cyan', attrs = ['bold']))

run_list.append('5. Process Tasks and Get Agent Info From Sisu')

TASK COUNT
0 Current Tasks
198 New Tasks
198 Total Tasks
SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.


In [11]:
# Cell Name: Insert Tasks

'''Once the SQL query for the agent information is copied to clipboard, run this cell'''

if 'df_reset_2' in locals():
    pass
else:
    df_reset_2 = df

# This cell takes in the agent information from the SQL query and creats a table to map values
df_assign_map_general = pd.DataFrame.from_dict({
    'first_name' : ['', '', '', '', ''],
    'last_name' : ['', '', '', '', ''],
    'agent_id' : ['T', 'A', 'I', 'T', 'A'],
    'name' : ['TC', 'Agent', 'ISA', 'RECRUITER COORDINATOR', 'RECRUITER (recruit platform)'],
    'agent_key' : ['T', 'A', 'I', 'T', 'A']
})

df_assign_map = pd.read_clipboard()
if df_assign_map.columns == '0':
    df_assign_map = pd.DataFrame(columns = ['first_name', 'last_name', 'agent_id'])
df_assign_map['name'] = df_assign_map['first_name'].str.strip() + " " + df_assign_map['last_name'].str.strip()
df_assign_map['agent_key'] = 'A'
df_assign_map['agent_key'] = df_assign_map['agent_key'] + df_assign_map['agent_id'].astype(str)
df_assign_map = df_assign_map.append(df_assign_map_general)

df = df.merge(df_assign_map, left_on = 'Assign to TC, Agent or assignee full name', right_on = 'name', how = 'left')
df = df.rename(columns = {'agent_key' : 'assign_to'})

df = df.reset_index(drop = True)
df['display_order'] = df.index
df['related_client_date_column'] = ''
df['client_type_id'] = ''

df_client_task_blueprints = df[client_task_blueprint_cols]
df_client_task_blueprints = df_client_task_blueprints.fillna('')

# Create SQL insert statement, and save to clipboard


task_blueprint_values = "("+df_client_task_blueprints['Team ID'].astype(str)+", '"+ df_client_task_blueprints['Task Name']+"', '"+ df_client_task_blueprints['Task Description']+"', '"+ df_client_task_blueprints['Task or Notification?']+"', "+ df_client_task_blueprints['display_order'].astype(str)+", '"+ df_client_task_blueprints['related_client_date_column']+ "', "+df_client_task_blueprints['Days'].astype(str)+", '"+ df_client_task_blueprints['Status']+"', '"+ df_client_task_blueprints['client_type_id']+"', "+ df_client_task_blueprints['created_ts']+", "+ df_client_task_blueprints['updated_ts']+", '"+ df_client_task_blueprints['assign_to']+ "'),"
df_client_task_blueprints['task_blueprint_values']  = task_blueprint_values
task_blueprints_insert_statement = 'INSERT INTO client_task_blueprint ("team_id","name","dscr","task_type","display_order","related_client_date_column","due_days","status","client_type_id","created_ts","updated_ts","assign_to") \nVALUES'

string = df_client_task_blueprints['task_blueprint_values'].to_string(index = False, header = False)
while "  " in string:
    string = string.replace("  ", " ")


clipboard.copy(task_blueprints_insert_statement + "\n" +  string.replace("\n (", "\n(").strip() + ";")

print(colored(f"Inserting Task Blueprints from the Team {team_id} template", 'white', attrs = ['bold']))
print(colored(f"An INSERT statement for the Task Bllueprints from the team {team_id} template has been copied to your clipboard. \nPaste into the Sisu Raw Data Tool \nThis will load the Task Blueprints from the template into Sisu.", 'green', attrs=['bold']))

run_list.append('6. Insert Tasks')


Inserting Task Blueprints from the Team 16032 template
An INSERT statement for the Task Bllueprints from the team 16032 template has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Task Blueprints from the template into Sisu.


In [10]:
df['Task Name'] = df['Task Name'].str.replace('d  4', 'd4')

## 4 <span style="color:#44d47d"> Matchup

In [13]:
# Cell Name: Get Task List Matchup

matchup_sql_text_1 = f"select task_list_id, name from client_task_list where team_id = {team_id} AND status = 'N'  ORDER BY task_list_id;"
df_matchup_sql_text_1 = pd.DataFrame([matchup_sql_text_1])
df_matchup_sql_text_1.to_clipboard(index=False,header=False)

print(colored(f"Collecting task_id and name from client_task_list in Sisu for team {team_id}", 'white', attrs = ['bold']))
print(colored("SQL query has been copied to your clipboard. \nPaste this into Sisu's Raw Data Tool", 'green', attrs=['bold']))
print(colored("Copy the returned table from the Raw Data Tool and run the following cell.", 'cyan', attrs = ['bold']))

run_list.append("7. Get Task List Matchup")

SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.


In [14]:
# Cell Name: Get Task Matchup

'''
Once the SQL return is copied to your clipboard, run this cell

MAKE SURE TO NOT COPY THE FOLLOWING SPACE  - WILL RESULT IN INCORRECT DELINIATION

'''

df_matchup_task_lists = pd.read_clipboard()
# df_matchup_task_lists = df_matchup_task_lists[df_matchup_task_lists['task_list_id'].notna()]

matchup_sql_text_2 = f"select task_blueprint_id, name from client_task_blueprint where team_id = {team_id} and status = 'N' order by task_blueprint_id;"
df_matchup_sql_text_2 = pd.DataFrame([matchup_sql_text_2])
df_matchup_sql_text_2.to_clipboard(index=False,header=False)

print(colored(f"Collecting task_blueprint_id and name from client_task_blueprint in Sisu for team {team_id}", 'white', attrs = ['bold']))
print(colored("SQL query has been copied to your clipboard. \nPaste this into Sisu's Raw Data Tool", 'green', attrs=['bold']))
print(colored("Copy the returned table from the Raw Data Tool and run the following cell.", 'cyan', attrs = ['bold']))

display(df_matchup_task_lists)

run_list.append('8. Get Task Matchup')

SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.


,task_list_id,name
0,25207,Pre Agent Onboarding
1,25208,New Agent
2,25209,Experienced Agent
3,25210,TJ New Agent
4,25211,TJ Experienced Agent
5,25212,Team Contract Signing
6,25213,Team Pre-Onboarding
7,25214,Team Onboarding


In [15]:
# Cell Name: Process and Insert Matchup

if 'df_reset_3' in locals():
    pass
else:
    df_reset_3 = df

'''Once SQL query is copied to clipboard, run this cell'''

df_matchup_task_blueprint = pd.read_clipboard()
# df_matchup_task_blueprint = df_matchup_task_blueprint[df_matchup_task_blueprint['task_blueprint_id'].notna()]

df_matchup_task_blueprint['task_blueprint_id'] = df_matchup_task_blueprint['task_blueprint_id'].astype(int)

df['Task Name'] = df['Task Name'].str.replace(r'\(', ' ').str.replace(r'\)',' ' ).str.replace('  ', ' ')
df_matchup_task_blueprint['name'] = df_matchup_task_blueprint['name'].str.replace(r'\(',  ' ').str.replace(r'\)',' ').str.replace('  ', ' ')



df = df.merge(df_matchup_task_lists, left_on = 'Task List Name', right_on = 'name', how = 'left')
df = df.merge(df_matchup_task_blueprint, left_on = 'Task Name', right_on = 'name', how = 'left')
# df = df[df['Task Name']]
df = df.reset_index(drop = True)

df_matchup = df[['task_list_id', 'task_blueprint_id']]
df_matchup['display_order'] = df.index

# Create SQL insert statement, and save to clipboard
task_list_blueprint_matchup_values = "("+df_matchup['task_list_id'].astype(str)+","+ df_matchup['task_blueprint_id'].astype(str)+","+ df_matchup['display_order'].astype(str)+ "),"
df_matchup['task_list_blueprint_matchup_values']  = task_list_blueprint_matchup_values



task_list_blueprints_matchup_insert_statement = 'INSERT INTO client_list_task_blueprint ("task_list_id","task_blueprint_id","display_order") \nVALUES'

string = df_matchup['task_list_blueprint_matchup_values'].to_string(index = False, header = False)
while "  " in string:
    string = string.replace("  ", " ")

clipboard.copy(task_list_blueprints_matchup_insert_statement + "\n" +  string.replace("\n (", "\n(").strip()[:-1] + ";")
# clipboard.copy(task_list_blueprints_matchup_insert_statement + "\n" +  df_matchup['task_list_blueprint_matchup_values'].to_string(index = False, header = False).strip().replace('   ', '  ').replace('  ', ' ').replace(' (', '(')+ ";")


print(colored(f"Inserting Matchup data for Team {team_id}", 'white', attrs = ['bold']))
print(colored(f"An INSERT statement for the Matchup data for Team {team_id} has been copied to your clipboard. \nPaste into the Sisu Raw Data Tool \nThis will load the Matchup data into Sisu.", 'green', attrs=['bold']))

run_list.append('9. Process and Insert Matchup')

Inserting Matchup data for Team 16032
An INSERT statement for the Matchup data for Team 16032 has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Matchup data into Sisu.


In [38]:
# Cell Name: Summary

summary_data = {
    'Initial Task List Count' : [len(current_task_list_names)],
    'New Task List Count' : [len(df['Task List Name'].unique())],
    'Total Task List Count': [len(current_task_list_names) + len(df['Task List Name'].unique())],
    'Initial Task Count' : [len(current_task_blueprint)],
    'New Task Count': [len(df)],
    'Total Task Count': [len(current_task_blueprint) + len(df)]
}
df_summary = pd.DataFrame.from_dict(summary_data)

print(colored("SUMMARY", 'cyan'))
print(colored("The output below (in white) is markdown format, which means copying and pasting into JIRA should load as a table just like the one in the following cell.", 'green'))
clipboard.copy(df_summary.T.reset_index().rename(columns = {'index':'Subject', 0:'Count'}).to_markdown(index = False))
print(df_summary.T.reset_index().rename(columns = {'index':'Subject', 0:'Count'}).to_markdown(index = False))

run_list.append('10. Summary')

SUMMARY
The output below (in white) is markdown format, which means copying and pasting into JIRA should load as a table just like the one in the following cell.
| Subject                 |   Count |
|:------------------------|--------:|
| Initial Task List Count |       0 |
| New Task List Count     |       8 |
| Total Task List Count   |       8 |
| Initial Task Count      |       0 |
| New Task Count          |     198 |
| Total Task Count        |     198 |


| Subject                 |   Count |
|:------------------------|--------:|
| Initial Task List Count |       0 |
| New Task List Count     |       8 |
| Total Task List Count   |       8 |
| Initial Task Count      |       0 |
| New Task Count          |     198 |
| Total Task Count        |     198 |

## FINISH